In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.datasets import fetch_california_housing
from sklearn.ensemble import HistGradientBoostingRegressor
import statistics as sts
from sliceline.slicefinder import Slicefinder
import optbinning
from dt import *
import psycopg2

(CVXPY) Jan 08 07:47:00 AM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.8.3296). Expected < 9.8.0. Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Jan 08 07:47:00 AM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.8.3296). Expected < 9.8.0. Please open a feature request on cvxpy to enable support for this version.')


## Dataset

We'll be testing out the Department of Transportation Airline On-Time Statistics dataset. We will set data sources to be years, which are 2018 to 2023. (2023 doesn't have November and December available yet.) The training columns will be:

1. Note: Year is excluded from training (by stationarity assumption)
2. Month (int)
3. Day (int)
4. Day of week (int)
5. Marketing airline (categorical, total 9): This is the airline which sold the ticket. 
6. Operating airline (categorical, total 21): This is the airline which operates the airplane. Could be the same as marketing airline, could be a different regional operator.
7. Origin & destination states (categorical, total 51)
8. Origin & destination airport coordinates (continuous): The original data is categorical with high number of options, it is probably more meaningful & tractable to extract the coordinates.
9. Distance (in miles)
10. Scheduled arrival & departure time (integer from 0000 to 2400). 

The prediction column is arrival delay (in minutes) which is encoded as integer, where negative is early and positive is late arrival. 

This dataset is very large: There are almost 40 million rows in total. 

This dataset is also very high-dimensional: there are 10 ordinal features and 132 categorical one-hot encoded features (30 airlines + 102 states). It is a good stress test for the system. 


 - NOTE: 2018-08 CSV is malformed and gives a parse error; needs fixing
 - NOTE: There may be issues where a flight has no arrival time due to cancellation, with unexpected handling of the arrival_delay column

**Reproducibility Notes**

The PyPi `sliceline` package requires Python 3.7~3.10.0, which is not the most up-to-date python version. Creating a virtual environment with python 3.9 should work. The conda environment should also have:

* sliceline
* pandas
* scikit
* optbinning

## Defining Reusable Methods

### Data Manipulation

In [2]:
def grab_data_psql(db_source):
    df = db_source.get_query_result()
    train_x, train_y = dt.split_xy(df, db_source.y_name)
    return train_x, train_y, df

### Model Training

In [3]:
def train_model(train_x, train_y):
    model = HistGradientBoostingRegressor(random_state=42)
    model.fit(train_x, train_y)
    return model

### Error Analysis

In [4]:
def get_errors(model, x, y):
    preds = model.predict(x)
    training_errors = (y - preds)**2
    return training_errors

In [5]:
def get_rms(arr):
    means = sts.mean(arr)
    rms = math.sqrt(means)
    return rms

In [6]:
def get_rms_error(model, x, y):
    errors = get_errors(model, x, y)
    return get_rms(errors)

### Binning

In [7]:
def bin_xs(train_x, train_errors):
    optimal_binner = optbinning.ContinuousOptimalBinning(max_n_bins=5)
    train_x_binned = pd.DataFrame(np.array(
        [
            optimal_binner.fit_transform(train_x[col], train_errors, metric="bins") for col in train_x.columns
        ]
    ).T, columns=train_x.columns)
    return train_x_binned

### Sliceliner

In [8]:
def get_slices(train_x_binned, train_errors, alpha = 0.9, k=1, max_l = 3, min_sup = 0, verbose = False):
    sf = Slicefinder(alpha = alpha, k = k, max_l = max_l, min_sup = min_sup, verbose = verbose)
    sf.fit(train_x_binned, train_errors)
    df = pd.DataFrame(sf.top_slices_, columns=sf.feature_names_in_, index=sf.get_feature_names_out())
    return df

In [9]:
# Reformat slices returned from sliceliner as dataframe into a matrix of strings
def reformat_slices(slice_df):
    slice_df.fillna('(-inf, inf)', inplace=True)
    slice_list = slice_df.values.tolist()
    slice_parsed = dt.parse_slices(slice_list)
    return slice_parsed

In [10]:
# Get the number of times each slice already exists in xs
def get_slice_cnts(xs, slices):
    cnts = []
    for slice_ in slices:
        cnt = 0
        for x in xs.values.tolist():
            if dt.belongs_to_slice(slice_, x):
                cnt += 1 
        cnts.append(cnt)
    return cnts

### Putting the Pipeline Together

In [11]:
# Train a model, report errors, and return model & binned train set
def pipeline_train(train_x, train_y, test_x, test_y):
    # Train model
    model = train_model(train_x, train_y)
    # Error analysis
    train_errors = get_errors(model, train_x, train_y)
    print("Train RMS error:", get_rms(train_errors))
    print("Test RMS error:", get_rms(get_errors(model, test_x, test_y)))
    # Binning
    train_x_binned = bin_xs(train_x, train_errors)
    return model, train_x_binned, train_errors

In [12]:
def pipeline_sliceline(train_x, train_x_binned, train_errors, alpha = 0.9, max_l = 3, min_sup = 0, k = 1):
    # Sliceliner
    slices_df = get_slices(train_x_binned, train_errors, alpha = 0.9, max_l = 3, min_sup = 0, verbose = False, k=k)
    slices = reformat_slices(slices_df)
    existing_cnts = get_slice_cnts(train_x, slices)
    print("Slices:")
    print(slices_df)
    print("Existing counts:", existing_cnts)
    return slices

In [13]:
# Obtain additional data
def pipeline_dt(sources, costs, slices, query_counts):
    dt = DT(sources, costs, slices, None, batch=100)
    additional_data = dt.run(query_counts)
    return additional_data

In [31]:
# Combine existing dataset with additional data
# Additional data is shuffled in
def pipeline_augment(train_x, train_y, additional_data, features):
    add_df = pd.DataFrame(additional_data, columns=features)
    add_x, add_y = dt.split_xy(add_df, 'arrival_delay')
    aug_x = pd.concat([train_x, add_x], ignore_index=True)
    aug_x = aug_x.sample(frac=1, random_state=12345)
    aug_y = pd.concat([train_y, add_y], ignore_index=True)
    aug_y = aug_y.sample(frac=1, random_state=12345)
    return aug_x, aug_y

## Flights Example

In [15]:
train_test_query_str = "SELECT f.month,f.day,f.weekday,a_origin.longitude AS origin_longitude,a_origin.latitude AS origin_latitude,a_dest.longitude AS dest_longitude,a_dest.latitude AS dest_latitude,f.distance,f.departure_scheduled,f.arrival_scheduled,f.arrival_delay FROM flights f JOIN airports a_origin ON f.origin_airport = a_origin.airport_code JOIN airports a_dest ON f.dest_airport = a_dest.airport_code WHERE f.arrival_delay IS NOT NULL ORDER BY random() LIMIT 10000;"

orig_dbsource = dt.DBSource('localhost','dtdemo','jwc','postgres',train_test_query_str,'arrival_delay')

# Yes, we will use simple uniform random subsampling even though it's bad practice for this proof of concept
# The dataset is very large so we can get away with it
# We also filter only rows that are not cancelled throughout this proof of concept
train_x, train_y, train = grab_data_psql(orig_dbsource)
train_x = dt.process_df(train_x)
test_x, test_y, test = grab_data_psql(orig_dbsource)
test_x = dt.process_df(test_x)
train_x

,month,day,weekday,origin_longitude,origin_latitude,dest_longitude,dest_latitude,distance,departure_scheduled,arrival_scheduled
0,2,10,5,-96.850833,32.845833,-102.201389,31.940833,319,1630,1745
1,6,15,5,-86.294722,39.717222,-115.158889,36.079722,1590,1433,1538
2,5,10,4,-84.428056,33.636667,-92.546944,31.324167,500,1628,1711
3,2,16,2,-118.408056,33.942500,-104.879722,39.774444,862,1144,1500
4,10,18,2,-87.396389,46.349167,-83.353333,42.212500,349,1300,1421
...,...,...,...,...,...,...,...,...,...,...
9995,9,11,7,-81.593056,38.372778,-87.906667,41.974444,416,1400,1445
9996,7,14,3,-123.217500,44.120833,-104.879722,39.774444,996,520,852
9997,4,26,1,-80.949167,35.213611,-76.201111,36.894722,290,2034,2156
9998,9,22,3,-116.222778,43.564444,-87.906667,41.974444,1437,1045,1513


In [16]:
print(len(train_x.columns))
print(len(test_x.columns))

10
10


In [17]:
model, train_x_binned, train_errors = pipeline_train(train_x, train_y, test_x, test_y)

Train RMS error: 39.923559441388825
Test RMS error: 56.14714618078793


In [18]:
slices = pipeline_sliceline(train_x, train_x_binned, train_errors, alpha = 0.5, max_l = 3, min_sup = 0, k = 5)

/home/jwc/miniconda3/envs/sliceline-venv/lib/python3.9/site-packages/sliceline/validation.py:457: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  and array.dtypes.apply(is_sparse).any()
/home/jwc/miniconda3/envs/sliceline-venv/lib/python3.9/site-packages/sliceline/validation.py:294: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/jwc/miniconda3/envs/sliceline-venv/lib/python3.9/site-packages/sliceline/validation.py:303: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/jwc/miniconda3/envs/sliceline-venv/lib/python3.9/site-packages/sliceline/slicefinder.py:329: RuntimeWarning: divide by zero encountered in divide
  (
/home/jwc/miniconda3/

Slices:
                month          day       weekday origin_longitude  \
slice_0   (-inf, inf)  (-inf, inf)   (-inf, inf)      (-inf, inf)   
slice_1  (-inf, 6.50)  (-inf, inf)   (-inf, inf)      (-inf, inf)   
slice_2   (-inf, inf)  (-inf, inf)   (-inf, inf)      (-inf, inf)   
slice_3  (-inf, 6.50)  (-inf, inf)   (-inf, inf)      (-inf, inf)   
slice_4   (-inf, inf)  (-inf, inf)  (-inf, 5.50)      (-inf, inf)   

        origin_latitude dest_longitude dest_latitude     distance  \
slice_0     (-inf, inf)    (-inf, inf)   (-inf, inf)  (-inf, inf)   
slice_1     (-inf, inf)    (-inf, inf)   (-inf, inf)  (-inf, inf)   
slice_2    [39.79, inf)    (-inf, inf)   (-inf, inf)  (-inf, inf)   
slice_3     (-inf, inf)    (-inf, inf)   (-inf, inf)  (-inf, inf)   
slice_4    [39.79, inf)    (-inf, inf)   (-inf, inf)  (-inf, inf)   

        departure_scheduled arrival_scheduled  
slice_0         (-inf, inf)    [1033.50, inf)  
slice_1         (-inf, inf)    [1033.50, inf)  
slice_2         (-

In [26]:
# Constants for defining data sources
# Data sources are divided by year
source_query_strs = [
    "SELECT f.month,f.day,f.weekday,a_origin.longitude AS origin_longitude,a_origin.latitude AS origin_latitude,a_dest.longitude AS dest_longitude,a_dest.latitude AS dest_latitude,f.distance,f.departure_scheduled,f.arrival_scheduled,f.arrival_delay FROM flights f JOIN airports a_origin ON f.origin_airport = a_origin.airport_code JOIN airports a_dest ON f.dest_airport = a_dest.airport_code WHERE f.arrival_delay IS NOT NULL AND year = 2018 ORDER BY random() LIMIT 10;",
    "SELECT f.month,f.day,f.weekday,a_origin.longitude AS origin_longitude,a_origin.latitude AS origin_latitude,a_dest.longitude AS dest_longitude,a_dest.latitude AS dest_latitude,f.distance,f.departure_scheduled,f.arrival_scheduled,f.arrival_delay FROM flights f JOIN airports a_origin ON f.origin_airport = a_origin.airport_code JOIN airports a_dest ON f.dest_airport = a_dest.airport_code WHERE f.arrival_delay IS NOT NULL AND year = 2019 ORDER BY random() LIMIT 10;",
    "SELECT f.month,f.day,f.weekday,a_origin.longitude AS origin_longitude,a_origin.latitude AS origin_latitude,a_dest.longitude AS dest_longitude,a_dest.latitude AS dest_latitude,f.distance,f.departure_scheduled,f.arrival_scheduled,f.arrival_delay FROM flights f JOIN airports a_origin ON f.origin_airport = a_origin.airport_code JOIN airports a_dest ON f.dest_airport = a_dest.airport_code WHERE f.arrival_delay IS NOT NULL AND year = 2020 ORDER BY random() LIMIT 10;",
    "SELECT f.month,f.day,f.weekday,a_origin.longitude AS origin_longitude,a_origin.latitude AS origin_latitude,a_dest.longitude AS dest_longitude,a_dest.latitude AS dest_latitude,f.distance,f.departure_scheduled,f.arrival_scheduled,f.arrival_delay FROM flights f JOIN airports a_origin ON f.origin_airport = a_origin.airport_code JOIN airports a_dest ON f.dest_airport = a_dest.airport_code WHERE f.arrival_delay IS NOT NULL AND year = 2021 ORDER BY random() LIMIT 10;",
    "SELECT f.month,f.day,f.weekday,a_origin.longitude AS origin_longitude,a_origin.latitude AS origin_latitude,a_dest.longitude AS dest_longitude,a_dest.latitude AS dest_latitude,f.distance,f.departure_scheduled,f.arrival_scheduled,f.arrival_delay FROM flights f JOIN airports a_origin ON f.origin_airport = a_origin.airport_code JOIN airports a_dest ON f.dest_airport = a_dest.airport_code WHERE f.arrival_delay IS NOT NULL AND year = 2022 ORDER BY random() LIMIT 10;",
    "SELECT f.month,f.day,f.weekday,a_origin.longitude AS origin_longitude,a_origin.latitude AS origin_latitude,a_dest.longitude AS dest_longitude,a_dest.latitude AS dest_latitude,f.distance,f.departure_scheduled,f.arrival_scheduled,f.arrival_delay FROM flights f JOIN airports a_origin ON f.origin_airport = a_origin.airport_code JOIN airports a_dest ON f.dest_airport = a_dest.airport_code WHERE f.arrival_delay IS NOT NULL AND year = 2023 ORDER BY random() LIMIT 10;"
]
sources = [dt.DBSource('localhost','dtdemo','jwc','postgres',query,'arrival_delay') for query in source_query_strs]
costs = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

In [27]:
# Arbitrary, for now
query_counts = [ 100, 100, 100, 100, 100 ]

In [28]:
additional_data = pipeline_dt(sources, costs, slices, query_counts)
additional_data

,month,day,weekday,origin_longitude,origin_latitude,dest_longitude,dest_latitude,distance,departure_scheduled,arrival_scheduled,median_house_value
0,12.0,20.0,4.0,-112.011667,33.434167,-121.940556,37.362778,621.0,1802.0,1852.0,-4
1,1.0,11.0,4.0,-81.308889,28.429444,-74.168611,40.692500,937.0,1110.0,1347.0,-6
2,2.0,16.0,5.0,-80.949167,35.213611,-115.158889,36.079722,1916.0,1459.0,1703.0,-7
3,10.0,27.0,6.0,-80.286111,25.792500,-76.201111,36.894722,802.0,2125.0,2354.0,-31
4,12.0,23.0,7.0,-81.308889,28.429444,-73.776944,40.638611,944.0,1729.0,2018.0,-18
...,...,...,...,...,...,...,...,...,...,...,...
333,12.0,16.0,7.0,-112.011667,33.434167,-104.879722,39.774444,602.0,1630.0,1810.0,7
334,2.0,23.0,5.0,-75.248611,39.868056,-122.375556,37.618889,2521.0,815.0,1140.0,5
335,9.0,13.0,4.0,-104.879722,39.774444,-119.767500,39.497778,804.0,825.0,945.0,0
336,10.0,28.0,7.0,-93.663056,41.533889,-87.906667,41.974444,299.0,1532.0,1659.0,-2


In [32]:
aug_x, aug_y = pipeline_augment(train_x, train_y, additional_data, train.columns)
aug_x

,month,day,weekday,origin_longitude,origin_latitude,dest_longitude,dest_latitude,distance,departure_scheduled,arrival_scheduled
1346,2.0,14.0,3.0,-75.440556,40.652222,-75.248611,39.868056,55.0,620.0,707.0
2184,2.0,10.0,1.0,-122.221667,37.718611,-116.222778,43.564444,512.0,2135.0,5.0
4718,1.0,15.0,1.0,-97.029722,32.894444,-157.920278,21.317778,3784.0,905.0,1349.0
1041,5.0,31.0,3.0,-80.149722,26.071667,-98.469444,29.533333,1145.0,1300.0,1521.0
6772,7.0,26.0,3.0,-76.670000,39.175000,-87.896944,42.946944,641.0,2240.0,2340.0
...,...,...,...,...,...,...,...,...,...,...
4478,2.0,4.0,4.0,-112.011667,33.434167,-98.469444,29.533333,843.0,2010.0,2334.0
4094,4.0,22.0,7.0,-119.718056,36.776111,-115.158889,36.079722,259.0,1348.0,1453.0
3492,9.0,30.0,1.0,-78.781944,35.875278,-73.776944,40.638611,427.0,1330.0,1517.0
2177,5.0,17.0,3.0,-96.914444,28.851111,-95.340000,29.983333,123.0,1605.0,1659.0
